<a href="https://colab.research.google.com/github/juromerop/TrabajosIAyMiniRobotsJuromerop/blob/main/Taller4/T4E4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4. Suponga que desea utilizar Programación Genética para encontrar el diseño de un circuito
lógico, tome como, ejemplo el codificador de 7 segmentos. Describa el conjunto de
terminales, el conjunto de funciones y la función de aptitud. Use una librería de Python.

In [25]:
import random
import operator

Definimos el conjunto de terminales (entradas binarias), definimos el conjunto de funciones (operaciones lógicas)

In [26]:
terminals = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

functions = {
    'AND': operator.and_,
    'OR': operator.or_,
    'NOT': operator.invert,
    'XOR': operator.xor
}

Función de aptitud,
Creamos una función que tome las entradas binarias y calcule la salida del circuito
Devolvemos un valor de aptitud que mida la diferencia entre la salida esperada y la real

In [27]:
def fitness(individual):
    try:
        expected_outputs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        actual_outputs = []

        num_inputs = len(terminals)
        for digit in range(10):
            inputs = [int(bit) for bit in format(digit, f'0{num_inputs}b')]
            output = evaluate_circuit(individual, inputs)
            actual_outputs.append(output)

        fitness_value = sum(abs(expected - actual) for expected, actual in zip(expected_outputs, actual_outputs))
        return fitness_value
    except ValueError:
        return 1e9

Función para evaluar el circuito lógico

In [28]:
def evaluate_circuit(individual, inputs):
    stack = []
    for gene in individual:
        if gene in terminals:
            stack.append(inputs[terminals.index(gene)])
        elif gene in functions:
            if gene == 'NOT':
                if len(stack) >= 1:
                    operand = stack.pop()
                    result = functions[gene](operand)
                    stack.append(result)
                else:
                    raise ValueError("Operación 'NOT' necesita un operando en la pila.")
            else:
                if len(stack) >= 2:
                    operand2 = stack.pop()
                    operand1 = stack.pop()
                    result = functions[gene](operand1, operand2)
                    stack.append(result)
                else:
                    raise ValueError("Operación necesita dos operandos en la pila.")
    if len(stack) == 1:
        return stack[0]
    else:
        raise ValueError("El diseño del circuito no es válido. Falta operar en algunos valores.")

Creamos la población inicial

In [29]:
def create_individual():
    chromosome_length = random.randint(10, 20)
    return [random.choice(terminals + list(functions.keys())) for _ in range(chromosome_length)]

population_size = 100
population = [create_individual() for _ in range(population_size)]

generations = 50
mutation_rate = 0.1

In [30]:
for generation in range(generations):
    fitness_scores = [fitness(individual) for individual in population]
    selected_indices = random.choices(range(population_size), k=population_size // 2, weights=fitness_scores)
    selected_population = [population[i] for i in selected_indices]

    new_population = []
    for _ in range(population_size // 2):
        parent1, parent2 = random.choices(selected_population, k=2)
        crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]

        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(child1) - 1)
            child1[mutation_point] = random.choice(terminals + list(functions.keys()))
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(child2) - 1)
            child2[mutation_point] = random.choice(terminals + list(functions.keys()))

        new_population.extend([child1, child2])

    population = new_population

Encontramos el diseno optimo

In [31]:
best_individual = min(population, key=fitness)
best_fitness = fitness(best_individual)

print("Mejor diseño:", best_individual)
print("Aptitud del mejor diseño:", best_fitness)

Mejor diseño: ['F', 'G', 'B', 'OR', 'NOT', 'E', 'XOR', 'NOT', 'C', 'D', 'OR', 'XOR', 'E', 'D', 'B']
Aptitud del mejor diseño: 1000000000.0
